In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from evoman.environment import Environment
from controller_custom import player_controller
from multiprocessing import Pool
from scipy.stats import ttest_ind, mannwhitneyu
import pandas as pd
import warnings

In [3]:
# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Algorithm A

In [20]:
def run_algorithm_a(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group):
	"""
	Runs Genetic Algorithm A with the provided environment and parameters.

	Parameters:
	- env: Evoman Environment instance.
	- npopulation: Population size.
	- gens: Number of generations.
	- mutation_rate: Mutation rate.
	- dom_u: Upper bound for gene values.
	- dom_l: Lower bound for gene values.
	- elitism_count: Number of elites to preserve.

	Returns:
	- history_mean: List of mean fitness per generation.
	- history_max: List of max fitness per generation.
	- best_solution: Best solution found.
	"""
	
	n_hidden_neurons = 10
	n_vars = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5
	

	# Run the simulation and return the fitness
	def simulate(x):
		env.player_controller.set(x, env.get_num_sensors())
		f, _, _, _ = env.play(pcont=x)
		return f
	
	# Evaluate the current population
	def evaluate(population):
		return np.array([simulate(individual) for individual in population])

	# Tournament Selection
	def tournament_selection(population, fitness, k=5):
		selected = []
		for _ in range(len(population)):
			contenders = np.random.choice(len(population), k, replace=False)
			winner = contenders[np.argmax(fitness[contenders])]
			selected.append(population[winner])
		return np.array(selected)

	# Uniform Crossover
	def crossover(parent1, parent2):
		mask = np.random.rand(n_vars) < 0.5
		child1 = np.where(mask, parent1, parent2)
		child2 = np.where(mask, parent2, parent1)
		return child1, child2

	# Gaussian Mutation
	def mutate(child):
		for i in range(n_vars):
			if np.random.rand() < mutation_rate:
				child[i] += np.random.normal(0, 0.1)
				child[i] = np.clip(child[i], dom_l, dom_u)
		return child

	# Initialize population
	population = np.random.uniform(dom_l, dom_u, (npopulation, n_vars))
	fitness = evaluate(population)

	# Record fitness over generations
	history_mean = []
	history_max = []

	# Genetic Algorithm Loop
	for generation in range(1, gens + 1):
		# Selection
		selected = tournament_selection(population, fitness)

		# Crossover
		offspring = []
		for i in range(0, npopulation, 2):
			parent1, parent2 = selected[i], selected[i+1]
			child1, child2 = crossover(parent1, parent2)
			offspring.extend([child1, child2])
		offspring = np.array(offspring)[:npopulation]

		# Mutation
		offspring = np.array([mutate(child) for child in offspring])

		# Evaluation
		offspring_fitness = evaluate(offspring)

		# Replacement: Elitism (keep the best individual)
		best_idx = np.argmax(fitness)
		worst_idx = np.argmin(offspring_fitness)
		if fitness[best_idx] > offspring_fitness[worst_idx]:
			offspring[worst_idx] = population[best_idx]
			offspring_fitness[worst_idx] = fitness[best_idx]

		population, fitness = offspring, offspring_fitness

		# Record statistics
		history_mean.append(np.mean(fitness))
		history_max.append(np.max(fitness))

		# Logging
		print(f'Generation {generation}: Best Fitness = {history_max[-1]:.4f}, Mean Fitness = {history_mean[-1]:.4f}')

	# Get best solution
	best_idx = np.argmax(fitness)
	best_solution = population[best_idx]

	np.savetxt(os.path.join('results', f'best_solution_A_{enemy_group}.txt'), best_solution)

	return history_mean, history_max, best_solution

# Algorithm B

In [21]:
def run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group):
    """
    Runs Genetic Algorithm A with the provided environment and parameters.

    Parameters:
    - env: Evoman Environment instance.
    - npopulation: Population size.
    - gens: Number of generations.
    - mutation_rate: Mutation rate.
    - dom_u: Upper bound for gene values.
    - dom_l: Lower bound for gene values.
    - elitism_count: Number of elites to preserve.

    Returns:
    - history_mean: List of mean fitness per generation.
    - history_max: List of max fitness per generation.
    - best_solution: Best solution found.
    """
    pass

# Parameters 

In [22]:
# Experiment Parameters

enemy_group_1 = [1, 4, 5, 8]
enemy_group_2 = [2, 3, 6, 7]

num_runs = 2 			# Set to 10
npopulation = 200
gens = 10 				# Set to 30
mutation_rate = 0.05
dom_u, dom_l = 1, -1
elitism_count_a = 1
elitism_count_b = 2
 

# The number of hidden neurons for the player controller
n_hidden_neurons = 10

# Initialize data storage
results = {
    'Algorithm': [],
    'Enemy Group': [],
    'Run': [],
    'Generation': [],
    'Mean Fitness': [],
    'Max Fitness': [],
    'Best Solution': []
}

best_solutions = {
    'Algorithm': [],
    'Enemy Group': [],
    'Run': [],
    'Best Solution': []
}

# Run Algorithm A on Enemy Group 1

In [23]:
print('Running Algorithm A for enemy group 1')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA1_EG1'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
				   		multiplemode="yes",
						enemies=enemy_group_1,
						playermode="ai",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_a(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_1')

	for gen in range(gens):
		results['Algorithm'].append('A')
		results['Enemy Group'].append(1)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('A')
	best_solutions['Enemy Group'].append(1)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA1_EG1.csv'), index=False)

Running Algorithm A for enemy group 1
Run 1

MESSAGE: Pygame initialized for simulation.
Generation 1: Best Fitness = 25.0588, Mean Fitness = -3.4824
Generation 2: Best Fitness = 25.0588, Mean Fitness = -0.2256
Generation 3: Best Fitness = 40.5695, Mean Fitness = 5.3919
Generation 4: Best Fitness = 40.5695, Mean Fitness = 11.1596
Generation 5: Best Fitness = 50.3529, Mean Fitness = 15.4967
Generation 6: Best Fitness = 50.3529, Mean Fitness = 18.4947
Generation 7: Best Fitness = 50.3529, Mean Fitness = 17.9141
Generation 8: Best Fitness = 55.8863, Mean Fitness = 20.5126
Generation 9: Best Fitness = 55.8863, Mean Fitness = 20.4211
Generation 10: Best Fitness = 55.8863, Mean Fitness = 21.6613
Run 2

MESSAGE: Pygame initialized for simulation.
Generation 1: Best Fitness = 34.6720, Mean Fitness = -1.5680
Generation 2: Best Fitness = 41.6832, Mean Fitness = 6.1931
Generation 3: Best Fitness = 42.0564, Mean Fitness = 10.7940
Generation 4: Best Fitness = 42.0564, Mean Fitness = 14.1206
Generat

# Run Algorithm A on Enemy Group 2

In [ ]:
print('Running Algorithm A for enemy group 2')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA1_EG2'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_2,
						playermode="ai",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_a(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_2')

	for gen in range(gens):
		results['Algorithm'].append('A')
		results['Enemy Group'].append(2)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('A')
	best_solutions['Enemy Group'].append(2)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA1_EG2.csv'), index=False)

# Run Algorithm B on Enemy Group 1

In [ ]:
print('Running Algorithm B for enemy group 1')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA2_EG1'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_1,
						playermode="ai",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_1')

	for gen in range(gens):
		results['Algorithm'].append('B')
		results['Enemy Group'].append(1)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('B')
	best_solutions['Enemy Group'].append(1)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA2_EG1.csv'), index=False)

# Run Algorithm B on Enemy Group 2

In [ ]:
print('Running Algorithm B for enemy group 2')

for run in range(1, num_runs +1):
	print(f'Run {run}')
	
	experiment_name = f'EA2_EG2'
	os.makedirs(experiment_name, exist_ok=True)

	env = Environment(experiment_name=experiment_name,
						enemies=enemy_group_2,
						playermode="ai",
						player_controller=player_controller(n_hidden_neurons),
						enemymode="static",
						level=2,
						speed="fastest",
						visuals=False)
	
	mean, max, best = run_algorithm_b(env, npopulation, gens, mutation_rate, dom_u, dom_l, enemy_group='enemy_group_2')

	for gen in range(gens):
		results['Algorithm'].append('B')
		results['Enemy Group'].append(2)
		results['Run'].append(run)
		results['Generation'].append(gen)
		results['Mean Fitness'].append(mean[gen])
		results['Max Fitness'].append(max[gen])
		results['Best Solution'].append(best)

	best_solutions['Algorithm'].append('B')
	best_solutions['Enemy Group'].append(2)
	best_solutions['Run'].append(run)
	best_solutions['Best Solution'].append(best)

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join('results', f'results_EA2_EG2.csv'), index=False)